分析用戶在網站上的瀏覽模式，特別是瞭解哪些頁面或功能經常一起被訪問。

In [104]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from scipy import sparse
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

get_link_list 抓取網頁，使用BeautifulSoup進行解析，並從超連結（<\a> 標籤）中提取所有文本，然後將這些文本作為列表返回。

get_journey 生成隨機網絡行程： 此函數生成一個隨機的稀疏矩陣，每一行代表用戶在網站的一次行程，每一列代表不同的超連結。density 參數控制每次行程中可能訪問的連結數量。


In [105]:

web_links = [
    'https://www.ezding.com.tw/cinemabooking?cinemaid=40288eb0050f542401050f554e7e0674',
    'https://www.ezding.com.tw/premiere'
]

#取得網頁文字
def get_link_list(link_id = 0):
    global web_links

    _web_page = urllib.request.urlopen(web_links[link_id])

    _soup = BeautifulSoup(_web_page, 'html.parser')

    _link_list = []

    for link in _soup.find_all('a'):
        _link_text = link.get_text().strip()

        if _link_text != _link_text:
            continue
        if _link_text == '':
            continue

        _link_list.append(_link_text)
        
    #print(_web_page)
    #print(_soup)
    return np.array(_link_list)

#生成隨機網路行程
def get_journey(link_list, num_journey = 100, ratio_foot = 0.10):     #num_journey = 模擬組數 
    return sparse.random(num_journey,                                 #sparse.random(行數,列數,非零元素的密度)
                         link_list.shape[0],
                         density = ratio_foot,
                         data_rvs = np.ones)


In [106]:
link_list = None

for link_id in range(len(web_links)):
    _link_list = get_link_list(link_id)
    if link_list is None:
        link_list = _link_list
    else:
        link_list = np.append(link_list, _link_list)

print("所有影城(站) :")
link_list[:5]

所有影城(站) :


array(['威秀影城Vieshow Cinema', '新光影城Skcinemas', 'in89豪華數位影城in89 Cinemax',
       '美麗華影城Miramar Cinemas', '樂聲影城Lux Cinemas'], dtype='<U22')

In [107]:
A = get_journey(link_list, num_journey=500, ratio_foot=0.5)

web_journey = []
for j in A.toarray():
    _journey = []

    for i in range(len(j)):
        if j[i]:
            _journey.append(link_list[i])
    web_journey.append(_journey)

#印出前幾筆
num_sample = 5
print("模擬路線前五筆 :")
for j in web_journey[:num_sample]:
    print(' >> '.join(j))

模擬路線前五筆 :
美麗華影城Miramar Cinemas >> 樂聲影城Lux Cinemas >> 真善美戲院Wonderful Theatre >> 新光影城Skcinemas >> 喜滿客影城Cinemark >> 真善美戲院Wonderful Theatre
威秀影城Vieshow Cinema >> 新光影城Skcinemas >> in89豪華數位影城in89 Cinemax >> 美麗華影城Miramar Cinemas >> 樂聲影城Lux Cinemas >> 喜滿客影城Cinemark >> in89豪華數位影城in89 Cinemax >> 美麗華影城Miramar Cinemas >> 樂聲影城Lux Cinemas >> 喜樂時代影城Century Asia >> 喜滿客影城Cinemark >> 真善美戲院Wonderful Theatre
威秀影城Vieshow Cinema >> in89豪華數位影城in89 Cinemax >> 美麗華影城Miramar Cinemas >> 樂聲影城Lux Cinemas >> 喜滿客影城Cinemark >> 威秀影城Vieshow Cinema >> 新光影城Skcinemas >> in89豪華數位影城in89 Cinemax >> 喜樂時代影城Century Asia
in89豪華數位影城in89 Cinemax >> 喜樂時代影城Century Asia >> 真善美戲院Wonderful Theatre >> 威秀影城Vieshow Cinema >> 新光影城Skcinemas >> 喜樂時代影城Century Asia >> 喜滿客影城Cinemark >> 真善美戲院Wonderful Theatre
威秀影城Vieshow Cinema >> 新光影城Skcinemas >> 美麗華影城Miramar Cinemas >> 樂聲影城Lux Cinemas >> 喜樂時代影城Century Asia >> 真善美戲院Wonderful Theatre >> 威秀影城Vieshow Cinema >> 新光影城Skcinemas >> 美麗華影城Miramar Cinemas >> 樂聲影城Lux Cinemas >> 真善美戲院Wonderful Theatre


使用 mlxtend 函式庫中的 TransactionEncoder 將網絡行程轉換為適合Apriori算法的格式。


In [108]:
te = TransactionEncoder()

te_array = te.fit(web_journey).transform(web_journey) #True, Flase矩陣, 標記有沒有進入此子網站

#print('\t'.join(te.columns_)) #站

te_dataset = pd.DataFrame(te_array, columns = te.columns_)

print(te_dataset.head(5))


   in89豪華數位影城in89 Cinemax  其他影城Others  喜樂時代影城Century Asia  喜滿客影城Cinemark  \
0                   False       False               False           True   
1                    True       False                True           True   
2                    True       False                True           True   
3                    True       False                True           True   
4                   False       False                True          False   

   威秀影城Vieshow Cinema  新光影城Skcinemas  樂聲影城Lux Cinemas  真善美戲院Wonderful Theatre  \
0               False           True             True                    True   
1                True           True             True                    True   
2                True           True             True                   False   
3                True           True            False                    True   
4                True           True             True                    True   

   美麗華影城Miramar Cinemas  
0                  True  
1   

應用Apriori算法： 使用 apriori 函數找出數據集中的常見項目集（一起被訪問的連結集合）。

最小支持度閾值 = 一個項目集至少出現在 百分之多少 的用戶行程中, 表示站跟站之間相關的比例有多高。

In [109]:
journey_rules = apriori(te_dataset, min_support=0.05, use_colnames=True)
journey_rules['length'] = journey_rules['itemsets'].apply(lambda x: len(x))

journey_length = 1
journey_2 = journey_rules[journey_rules['length']>journey_length] #超過兩個連結的項目集
print(journey_2.head())

print('共識別出 %d 種不同的瀏覽模式(足跡)，每種模式中至少包括 %d 個經常一起被訪問的網頁連結' % (journey_2.shape[0],journey_length+1))

    support                                      itemsets  length
9     0.536          (in89豪華數位影城in89 Cinemax, 其他影城Others)       2
10    0.560  (in89豪華數位影城in89 Cinemax, 喜樂時代影城Century Asia)       2
11    0.558       (in89豪華數位影城in89 Cinemax, 喜滿客影城Cinemark)       2
12    0.526  (in89豪華數位影城in89 Cinemax, 威秀影城Vieshow Cinema)       2
13    0.544       (新光影城Skcinemas, in89豪華數位影城in89 Cinemax)       2
共識別出 502 種不同的瀏覽模式(足跡)，每種模式中至少包括 2 個經常一起被訪問的網頁連結
